## Import

In [1]:
import numpy as np
import json
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import fastai
from fastai import *
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy
from fastai.optimizer import Adam
from fastai.callback.progress import ProgressCallback
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from parse_preprocessed_data import get_inputs_and_targets
from LSTM_Model2 import LSTMModel2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Hyper-Parameters

In [2]:
seq_length = 50

hidden_size = 128
learning_rate = 2e-3
dropout = 0.5
batch_size = 100
num_layers = 3
max_epochs = 20
validation_prop = 0.2

## Load Data

In [3]:
#Not Original
char_to_ix, ix_to_char, vocab_size, inputs, targets = get_inputs_and_targets('data_preprocessed/mario.txt', seq_length)
vocab_size, inputs.shape, targets.shape

#I know these inputs and targets to be correct, as this is the same as the original model
#vocab_size is 15
#inputs is (num_sequences, sequence_size, vocab_size), and is hot-shot encoded
#targets is (num_sequence, sequence_size)

Unique chars: ['\n', '-', '<', '>', '?', 'B', 'E', 'Q', 'S', 'X', '[', ']', 'b', 'o', 'x']
Number of unique chars: 15


  0%|          | 0/37 [00:00<?, ?it/s]

(15, (124700, 50, 15), (124700, 50))

In [4]:
#Not Original
#Unrelated for my issue
#first_three_cols = inputs[0][:3 * 17]
#np.savetxt('data_preprocessed/seed.txt', first_three_cols)

In [5]:
#Not Original
#Unrelated for my issue
with open('data_preprocessed/char_to_ix.json', 'w+') as json_f:
    json.dump(char_to_ix, json_f)

with open('data_preprocessed/ix_to_char.json', 'w+') as json_f:
    json.dump(ix_to_char, json_f)

In [6]:
#inputs.shape, inputs.dtype

In [7]:
#targets.shape, targets.dtype

In [8]:
#inputs = torch.from_numpy(inputs).type(torch.Tensor).to(device)
#targets = torch.from_numpy(targets).type(torch.Tensor).to(device)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [10]:
train_dataset = CustomDataset(inputs, targets)

## Model Callbacks

In [11]:
#Compute loss as the cross-entropy loss betweenthe predicted values and the true labels
def custom_loss(y_pred_tup, y_true):
    y_pred,_,_,_,_,_,_ = y_pred_tup
    return F.cross_entropy(
        y_pred.view(-1, vocab_size),
        y_true.long().view(-1)
    )

In [12]:
#Essentially tries to compute the accuracy of the predicted sequence to the true sequence
def custom_acc(y_pred_tup, y_true):
    y_pred,_,_,_,_,_,_ = y_pred_tup
    #Calculate predicted classes by taking the argmax along the second dimension
    #Reshape the predicted values into a 2D tensor with shape (batch_size * seq_length, vocab_size)
    pred_classes = torch.argmax(y_pred.view(-1, vocab_size), dim=1)
    # Flatten the true labels
    true_classes = y_true.view(-1)
    #Compare each element in the sequence, then converting the binary equality value to a float
    class_equality = torch.eq(pred_classes, true_classes).float()
    #The mean of all of the equality scores is the accuracy
    return torch.mean(class_equality)

## Model

In [13]:
# Initialize model
model = LSTMModel2(vocab_size, hidden_size, num_layers, dropout).to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(max_epochs):
    model.train()
    total_loss = 0.0
    
    for batch_inputs, batch_targets in train_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        
        # Forward pass
        outputs, _ = model(batch_inputs)
        
        # Reshape outputs and targets to have the same batch size
        outputs_flat = outputs.view(-1, outputs.size(-1))  # Flatten outputs
        targets_flat = batch_targets.view(-1)  # Flatten targets
        # Calculate loss
        loss = criterion(outputs_flat, targets_flat.long())
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        total_loss += loss.item() * batch_inputs.size(0)  # Accumulate loss for the entire batch
    
    # Calculate average loss for the epoch
    epoch_loss = total_loss / len(train_loader.dataset)
    
    # Print epoch-wise loss
    print(f'Epoch [{epoch+1}/{max_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/20], Loss: 0.5131
Epoch [2/20], Loss: 0.3058
Epoch [3/20], Loss: 0.2772
Epoch [4/20], Loss: 0.2609
Epoch [5/20], Loss: 0.2509
Epoch [6/20], Loss: 0.2442
Epoch [7/20], Loss: 0.2393
Epoch [8/20], Loss: 0.2350
Epoch [9/20], Loss: 0.2320
Epoch [10/20], Loss: 0.2290
Epoch [11/20], Loss: 0.2268
Epoch [12/20], Loss: 0.2247
Epoch [13/20], Loss: 0.2227
Epoch [14/20], Loss: 0.2213
Epoch [15/20], Loss: 0.2199
Epoch [16/20], Loss: 0.2183
Epoch [17/20], Loss: 0.2172
Epoch [18/20], Loss: 0.2162
Epoch [19/20], Loss: 0.2153
Epoch [20/20], Loss: 0.2141


## Train Model

In [17]:
#Not Original
seed = np.loadtxt('data_preprocessed/seed.txt', dtype=float)[:3*17 - 1].copy()

with open('data_preprocessed/ix_to_char.json', 'r') as json_f:
    ix_to_char = json.load(json_f)
    
with open('data_preprocessed/char_to_ix.json', 'r') as json_f:
    char_to_ix = json.load(json_f)

In [18]:
#Not Original
def onehot_to_string(onehot):
    ints = np.argmax(onehot, axis=-1)
    chars = [ix_to_char[str(ix)] for ix in ints]
    string = "".join(chars)
    char_array = []
    for line in string.rstrip().split('\n')[:-1]:
        if len(line) == 16:
            char_array.append(list(line))
        elif len(line) > 16:
            char_array.append(list(line[:16]))
        elif len(line) < 16:
            char_array.append(['-'] * (16 - len(line)) + list(line))
    char_array = np.array(char_array).T
    string = ""
    for row in char_array:
        string += "".join(row) + "\n"
    return string

In [19]:
#Not Original
seed[17+14] = 0
seed[17+14][char_to_ix['x']] = 1
seed[17*2+14] = 0
seed[17*2+14][char_to_ix['x']] = 1
print(onehot_to_string(seed))

--
--
--
--
--
--
--
--
--
--
--
--
--
--
-x
XX



In [20]:
#Not Original
def get_seed():
    seed = np.loadtxt('data_preprocessed/seed.txt', dtype=float)[:3*17 - 1]
    seed[17+14] = 0
    seed[17+14][char_to_ix['x']] = 1
    seed[17*2+14] = 0
    seed[17*2+14][char_to_ix['x']] = 1
    return seed

In [21]:
#Not Original
seed = get_seed()
seed.shape

(50, 15)

In [52]:
#Not Original
num_levels_to_gen = 10
num_chunks = 10
num_cols_per_chunk = 16
num_rows_per_col = 17
num_chars_to_gen = num_chunks * num_cols_per_chunk * num_rows_per_col - len(seed)
print(num_chars_to_gen)

2670


In [23]:
get_seed().shape

(50, 15)

In [123]:
# Initialize hidden states
h0 = torch.zeros(num_layers, 1, hidden_size).to(device)
c0 = torch.zeros(num_layers, 1, hidden_size).to(device)

seed = get_seed()
seed = np.expand_dims(seed, axis=0)
seed = np.repeat(seed, num_levels_to_gen, axis=0)
seed.shape

gen = []

for i in range(num_levels_to_gen):
    # Seed initialization
    seed = torch.from_numpy(get_seed()).type(torch.Tensor).to(device)
    generated_seq = []
    h0 = torch.zeros(num_layers, 1, hidden_size).to(device)
    c0 = torch.zeros(num_layers, 1, hidden_size).to(device)
    # Generate characters
    for j in range(num_chars_to_gen):
        # Forward pass through the model
        output, (h0, c0) = model(seed.view(1, -1, vocab_size), (h0, c0))
        output_probs = F.softmax(output, dim=-1).squeeze().cpu().detach().numpy()
    
        # Sample the next character
        next_char_idx = np.random.choice(vocab_size, p=output_probs[-1])
        next_char = ix_to_char[str(next_char_idx)]
        generated_seq.append(next_char)
        
        # Update the seed for the next iteration
        #seed = torch.cat((seed[1:], F.one_hot(torch.tensor(next_char_idx), num_classes=vocab_size).unsqueeze(0)), dim=0).to(device)
        next_char_tensor = torch.tensor(next_char_idx).to(device)
        one_hot_tensor = F.one_hot(next_char_tensor, num_classes=vocab_size).unsqueeze(0).to(device)
        seed = torch.cat((seed[1:].to(device), one_hot_tensor), dim=0)
        
    # Convert generated sequence to a string
    generated_str = ''.join(generated_seq)
    gen.append(generated_str)


In [124]:
gen[0]

'\n-------------xxX\n-------------xEX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n------S---S--xXX\n------S--ES--xXX\n------S---S--xXX\n------S---?--xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n-------------xXX\n------------xxXX\n-----------xx-XX\n----------xx----\n----------xX----\n-------X-xxX----\n-------X-x-E----\n-------X-x-E----\n-------X-x-X----\n----------xX----\n----------xX----\n----------xX----\n----------xX----\n----------xX----\n---------xxX----\n--------xx------\n--------x-------\n--------x-------\n-------ExxX-----\n-------xxX------\n------xx--------\n-----xx---------\n-----x-E-------X\n-----x----------\n------x---------\n-------x--------\n--------x-------\n---------x-Q----\n---------x------\n----------x-----\n-----------x----\n------------x---\n-------------xX-\n-------------xX-\n-------------xx-\n-------------xxX\n-------

In [134]:
modified_gen = []
for i in range(len(gen)):
    rows = gen[i].split('\n')[1:-1]
    for r in range(len(rows)):
        row = rows[r]
        # Pad the row with 'x' until it reaches length 16
        rows[r] = row.ljust(16, 'x')
    new_string = ''
    for j in range(16):
        new_string = new_string + ''.join([row[j] for row in rows])
        if j < 15:
            new_string += '\n'
    modified_gen.append(new_string)

In [135]:
#Not Original
for i, g in enumerate(modified_gen):
    with open(f'generated_levels_txt/{i+1}.txt', 'w+') as txt_f:
        txt_f.write(g)